# Diagrama UML

Libro
- id (PK)
- titulo
- precio
- rating
- in_stock
- categoria_id (FK)

Autor
- id (PK)
- nombre

Libro_Autor
- libro_id (FK)
- autor_id (FK)

Categoria
- id (PK)
- nombre

# Esquema SQL

## DDL

CREATE TABLE Categoria (   
- id SERIAL PRIMARY KEY,  
- nombre TEXT NOT NULL  
);


CREATE TABLE Autor (  
- id SERIAL PRIMARY KEY,  
- nombre TEXT NOT NULL  
);  

CREATE TABLE Libro (  
- id SERIAL PRIMARY KEY,  
- titulo TEXT NOT NULL,  
- precio NUMERIC(6, 2) NOT NULL,  
- rating INTEGER NOT NULL,  
- in_stock BOOLEAN NOT NULL,  
- categoria_id INTEGER NOT NULL,  
- FOREIGN KEY (categoria_id) REFERENCES Categoria(id)  
);  

CREATE TABLE Libro_Autor (  
- libro_id INTEGER NOT NULL,  
- autor_id INTEGER NOT NULL,  
- PRIMARY KEY (libro_id, autor_id),  
- FOREIGN KEY (libro_id) REFERENCES Libro(id),  
- FOREIGN KEY (autor_id) REFERENCES Autor(id)  
);

In [ ]:
%pip install requests
%pip install beautifulsoup4
%pip install sqlalchemy
%pip install pandas

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Numeric, Boolean, ForeignKey, Table
from sqlalchemy.orm import relationship, declarative_base, sessionmaker
from contextlib import contextmanager

# Base para las clases
Base = declarative_base()

# Crear motor SQLite con configuración optimizada porque jupy
engine = create_engine(
    "sqlite:///books.db",
    echo=False,
    connect_args={"check_same_thread": False},  # Necesario para jupyter
    pool_pre_ping=True  # Verifica conexiones antes de usarlas
)

# Tabla intermedia Libro_Autor (relación muchos a muchos)
libro_autor = Table(
    'libro_autor', Base.metadata,
    Column('libro_id', Integer, ForeignKey('libro.id'), primary_key=True),
    Column('autor_id', Integer, ForeignKey('autor.id'), primary_key=True)
)

class Categoria(Base):
    __tablename__ = 'categoria'
    id = Column(Integer, primary_key=True)
    nombre = Column(String, nullable=False)

    
class Autor(Base):
    __tablename__ = 'autor'
    id = Column(Integer, primary_key=True)
    nombre = Column(String, nullable=False)


class Libro(Base):
    __tablename__ = 'libro'
    id = Column(Integer, primary_key=True)
    titulo = Column(String, nullable=False)
    precio = Column(Numeric(6, 2), nullable=False)
    rating = Column(Integer, nullable=False)
    in_stock = Column(Boolean, nullable=False)
    categoria_id = Column(Integer, ForeignKey('categoria.id'), nullable=False)

    # Relaciones
    categoria = relationship("Categoria")
    autores = relationship("Autor", secondary=libro_autor, backref="libros")

# Crear todas las tablas
Base.metadata.create_all(engine)

# manejamos las sessiones
@contextmanager
def get_session():
    Session = sessionmaker(bind=engine)
    session = Session()
    try:
        yield session
        session.commit()
    except Exception:
        session.rollback()
        raise
    finally:
        session.close()


In [ ]:
# Extracción de datos con verificación de existencia
def safe_extract(selector, soup_detalle, attribute=None):
    element = soup_detalle.select_one(selector)
    if not element:
        return None
    return element.text if attribute is None else element.get(attribute)

# Encontrar el autor con el titulo y categoria usando APIs
def get_author_from_apis_books(title, category):
    query = f'intitle:"{title}"'
    query += f' subject:"{category}"'

    try:
        # Intentamos primero con Open Library 
        response = requests.get(
            "https://openlibrary.org/search.json",
            params={"title": title, "limit": 1}
        )
        ol_data = response.json()
        if ol_data.get("docs") and ol_data["docs"][0].get("author_name"):
            return ol_data["docs"][0]["author_name"][0]
        
        # Si no, buscamos en Google books
        response = requests.get(
            "https://www.googleapis.com/books/v1/volumes",
            params={
                "q": query,
                "maxResults": 1,
                "orderBy": "relevance",
                "langRestrict": "en"}
        )
        data = response.json()
        
        if data.get("items"):
            authors = data["items"][0]["volumeInfo"].get("authors")
            if authors and len(authors) > 0:
                return authors[0]

        
    except Exception as e:
        print(f"Error al consultar APIs Books: {e}")

    return 'Desconocido'


In [ ]:
def scrape_and_save_book (libro):

    # Extraer URL del detalle 
    detalle_rel_url = libro.h3.a["href"]
    if detalle_rel_url.startswith("../../../"):
        detalle_rel_url = detalle_rel_url.replace("../../../", "catalogue/")
    elif detalle_rel_url.startswith("../../"):
        detalle_rel_url = detalle_rel_url.replace("../../", "catalogue/")
    elif detalle_rel_url.startswith("../"):
        detalle_rel_url = detalle_rel_url.replace("../", "catalogue/")

    detalle_url = "https://books.toscrape.com/" + detalle_rel_url

    # Verificar URL antes de hacer la solicitud
    print("\n" + "="*50)
    print(f"URL de detalle a acceder: {detalle_url}")

    # Configurar headers para parecer un navegador real
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Hacer la solicitud con manejo de errores
    try:
        resp_detalle = requests.get(detalle_url, headers=headers, timeout=10)
        resp_detalle.raise_for_status()  # Lanza error si la respuesta no es 200
    except requests.exceptions.RequestException as e:
        print(f"Error al acceder a {detalle_url}: {e}")
        raise

    soup_detalle = BeautifulSoup(resp_detalle.text, "html.parser")


    # Precio
    precio_element = soup_detalle.select_one("p.price_color")
    if not precio_element:
        # Buscar alternativas si el selector principal falla
        precio_element = soup_detalle.select_one(".product_main .price_color")
        
    if precio_element:
        precio = float(precio_element.text.replace("Â£", ""))
    else:
        precio = None
        print("Advertencia: No se encontró el precio del libro")

    # Resto de los datos con funcion manejo seguro
    titulo = safe_extract(".product_main h1", soup_detalle)
    stock_texto = safe_extract("p.instock.availability", soup_detalle)
    in_stock = stock_texto and "In stock" in stock_texto

    # Rating
    rating_tag = soup_detalle.select_one("p.star-rating")
    rating = 0
    if rating_tag:
        rating_clases = rating_tag.get("class", [])
        if len(rating_clases) > 1:
            rating_clase = rating_clases[1]
            mapa_rating = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
            rating = mapa_rating.get(rating_clase, 0)

    # Categoría
    categoria_element = soup_detalle.select("ul.breadcrumb li a")
    categoria = categoria_element[-1].text if categoria_element else None

    # Autor
    autor = get_author_from_apis_books(titulo, categoria)

    # Insertar en la base de datos
    with get_session() as session:

        try:
            # Obtener o crear categoría
            categoria_obj = session.query(Categoria).filter_by(nombre=categoria).first()
            if not categoria_obj:
                categoria_obj = Categoria(nombre=categoria)
                session.add(categoria_obj)
                session.flush()  # Para obtener el ID
            
            # Obtener o crear autor
            autor_obj = session.query(Autor).filter_by(nombre=autor).first()
            if not autor_obj:
                autor_obj = Autor(nombre=autor)
                session.add(autor_obj)
                session.flush()
            
            # Crear el libro
            libro = Libro(
                titulo=titulo,
                precio=precio,
                rating=rating,
                in_stock=in_stock,
                categoria_id=categoria_obj.id
            )
            
            # Añadir la relación muchos-a-muchos con el autor
            libro.autores.append(autor_obj)
            
            session.add(libro)
            
            print("Libro insertado correctamente:")
            print("Título:", titulo)
            print("Precio:", precio)
            print("Stock:", in_stock)
            print("Rating:", rating)
            print("Categoría:", categoria)
            print("Autor:", autor)

            return libro

        except Exception as e:
            print(f"Error al insertar el libro {titulo}: {e}")
            raise


In [ ]:
import requests
from bs4 import BeautifulSoup

# Obtener lista de libros de la página principal
base_url = "https://books.toscrape.com/catalogue/category/books_1/page-{}.html"

try: 

    for page in range(1, 51):
        url = base_url.format(page)
           
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # Creamos nuestra variable donde estan almacewnados todos los libros y verificamos que todo este correcto
        libros = soup.select('article.product_pod')
        if not libros:
            raise ValueError("No se encontraron libros en esta principal")
        else:
            print(f'\nEncontrados {len(libros)} libros.\n')

        for libro_element in libros:
            scrape_and_save_book(libro_element)

finally:
    # Limpieza final
    engine.dispose()
    print('\nProceso completado')

In [ ]:
import pandas as pd

# 1- Libros con mas de 3 estrellas y a menos de £12
%time
query1 = """
SELECT titulo, precio, rating
FROM libro
WHERE rating > 3 AND precio < 12;
"""
df1 = pd.read_sql(query1, engine)
df1

CPU times: total: 0 ns
Wall time: 0 ns


,titulo,precio,rating
0,Superman Vol. 1: Before Truth (Superman by Gen...,11.89,5
1,Old School (Diary of a Wimpy Kid #10),11.83,5
2,I Am Pilgrim (Pilgrim #1),10.60,4
3,Greek Mythic History,10.23,5
4,Dear Mr. Knightley,11.21,5
5,City of Fallen Angels (The Mortal Instruments #4),11.23,4
6,"The Sleep Revolution: Transforming Your Life, ...",11.68,4
7,"NaNo What Now? Finding your editing process, r...",10.41,4
8,History of Beauty,10.29,4
9,The Origin of Species,10.01,4


In [16]:
# 2- Libros con precio > 50
%time
query2 = """
SELECT titulo, precio
FROM libro
WHERE precio > 50;
"""
df2 = pd.read_sql(query2, engine)
df2.head()

CPU times: total: 0 ns
Wall time: 0 ns


,titulo,precio
0,A Light in the Attic,51.77
1,Tipping the Velvet,53.74
2,Soumission,50.10
3,Sapiens: A Brief History of Humankind,54.23
4,The Black Maria,52.15


In [ ]:
# 3. Autores con mas libros
%time
query3 = """
SELECT a.nombre, COUNT(*) AS total_libros
FROM autor a
JOIN libro_autor la ON a.id = la.autor_id
GROUP BY a.id
ORDER BY total_libros DESC;
"""
df3 = pd.read_sql(query3, engine)
df3.head()

CPU times: total: 0 ns
Wall time: 0 ns


,nombre,total_libros
0,Desconocido,147
1,Stephen King,9
2,Worth Books,9
3,Sophie Kinsella,7
4,Cassandra Clare,5


In [20]:
# 4. Categorías con mas de 20 libros
%time
query4 = """
SELECT c.nombre, COUNT(*) AS total_libros
FROM categoria c
JOIN libro l ON c.id = l.categoria_id
GROUP BY c.id
HAVING COUNT(*) > 20
ORDER BY total_libros DESC;
"""
df4 = pd.read_sql(query4, engine)
df4.head()

CPU times: total: 0 ns
Wall time: 0 ns


,nombre,total_libros
0,Default,152
1,Nonfiction,110
2,Sequential Art,75
3,Add a comment,67
4,Fiction,65


In [22]:
# 5. Libros en stock con rating máximo
%time
query5 = """
SELECT titulo, rating, in_stock
FROM libro
WHERE in_stock > 0
ORDER BY rating DESC;
"""
df5 = pd.read_sql(query5, engine)
df5.head()

CPU times: total: 0 ns
Wall time: 0 ns


,titulo,rating,in_stock
0,Sapiens: A Brief History of Humankind,5,1
1,Set Me Free,5,1
2,Scott Pilgrim's Precious Little Life (Scott Pi...,5,1
3,Rip it Up and Start Again,5,1
4,Chase Me (Paris Nights #2),5,1
